# Merging and cleaning overall audio feature df

In [1]:
# impoarting needed libraries

import pandas as pd

In [3]:
# importing both audio feature data files

audio_features_top100 = pd.read_csv('../data/top100_audio_features.csv')
audio_features_dr_sample = pd.read_csv('../data/dr_sample_audio_features.csv')

In [4]:
# checking columns top100

audio_features_top100.columns

# need to remove the Unnamed

Index(['Unnamed: 0', 'id', 'track_title', 'track_explicit', 'track_artist',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [5]:
# checking columns dr sample

audio_features_dr_sample.columns

# need to remove the Unnamed and 0 column

Index(['Unnamed: 0', 'id', 'track_title', 'track_explicit', 'track_artist',
       '0', 'acousticness', 'analysis_url', 'danceability', 'duration_ms',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'track_href', 'type', 'uri',
       'valence'],
      dtype='object')

In [7]:
len(audio_features_top100.columns), len(audio_features_dr_sample.columns)

(22, 23)

to dos:
- remove Unnamed column from both df
- remove 0 column from dr df
- rearrange df dr columns to same order as top100 dr
- add column data source (dr sample, top100)
- add genre in both
- concat both dfs to one

In [9]:
# dropping not needed columns

audio_features_top100 = audio_features_top100.drop(['Unnamed: 0'], axis = 1)
audio_features_dr_sample = audio_features_dr_sample.drop(['Unnamed: 0'], axis = 1)
audio_features_dr_sample = audio_features_dr_sample.drop(['0'], axis = 1)

In [10]:
# adding column datasource

audio_features_top100['data_source'] = 'top100'
audio_features_dr_sample['data_source'] = 'dr_sample'

In [13]:
# adding column genre in again dr sample

audio_features_dr_sample['genre'] = 'deutschrap'

In [17]:
# adding column genre in again top100

# importing dr artist data
dr_artists = pd.read_csv('../data/dr_artist_name_id_df.csv')

# creating a list of all unique dr artist names
dr_artists_lst = list(dr_artists['artist_name'].unique())

# importing top20 hip hop data
top20_hiphop = pd.read_csv('../data/top20_hiphop_tracks_raw.csv')

# creating a list of all unique hiphop artist names
top20_hiphop_artists_lst = list(top20_hiphop['artist'].unique())

# importing christmas tracks
christmas_tracks = pd.read_csv('../data/christmas_tracks.csv')

# creating a list of all unique christmas tracks
christmas_tracks_lst = list(christmas_tracks['title'].unique())

audio_features_top100['genre'] = 0

# creating new feature to mark the tracks which are christmas tracks
for track in christmas_tracks_lst:
    for i in range(len(audio_features_top100)):
        if track in audio_features_top100.loc[i,'track_title']:
            audio_features_top100.loc[i, 'genre'] = 'christmas'

# filling empty genre column in top100 with hiphop from artists in top20
for artist in top20_hiphop_artists_lst:
    for i in range(len(audio_features_top100)):
        if artist in audio_features_top100.loc[i,'track_artist']:
            audio_features_top100.loc[i, 'genre'] = 'hiphop'
            
# filling empty genre column in top100 with dr artists from artists in dr

for artist in dr_artists_lst:
    for i in range(len(audio_features_top100)):
        if artist in audio_features_top100.loc[i,'track_artist']:
            audio_features_top100.loc[i, 'genre'] = 'deutschrap'

In [19]:
audio_features_top100.columns

Index(['id', 'track_title', 'track_explicit', 'track_artist', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'data_source', 'genre'],
      dtype='object')

In [20]:
# rearranging the columns

audio_features_dr_sample = audio_features_dr_sample[['id', 'track_title', 'track_explicit', 'track_artist', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'data_source', 'genre']]

In [22]:
len(audio_features_top100), len(audio_features_dr_sample)

(17735, 28077)

In [23]:
# concatenating the 2 df

audio_features_all = pd.concat([audio_features_dr_sample, audio_features_top100], axis=0)
len(audio_features_all)

45812

In [24]:
# saving df 

audio_features_all.to_csv('../data/audio_features_all.csv')